# Решение задач с занятия №3

Чтобы все работало, нужно сначала запустить все ячейки из ноутбука 03-databases, чтобы создать базу с таблицами subjects, teachers и subject_to_teachers

In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
sql = sqlite3.connect('../data/school.db')

## Задание 1

Добавить таблицу "students" и "groups", зачислить в студентов в группы

In [19]:
group_numbers = [101, 102]
student_names = ["Alice", "Bob", "Charlie", "Diana"]
group_ids = [0, 1, 0, 1]

students = pd.DataFrame({'name': student_names, 'group_id': group_ids})
groups = pd.DataFrame({'number': group_numbers})

In [20]:
students.to_sql('students', sql, index_label='id', if_exists='replace')
groups.to_sql('groups', sql, index_label='id', if_exists='replace')

Проверяем

In [21]:
pd.read_sql('SELECT s.name, g.number FROM students as s JOIN groups as g ON g.id = s.group_id', sql)

,name,number
0,Alice,101
1,Bob,102
2,Charlie,101
3,Diana,102


## Задание 2

Добавить таблицу "marks"

Посчитать средний балл всех студентов

Посчитать среднюю оценку каждого студента по всем предметам

In [13]:
student_id = [0, 0, 0, 1, 1, 1, 2, 2, 2]
subject_id = [0, 0, 1, 0, 0, 1, 0, 0, 1]
marks = [3, 4, 5, 3, 5, 4, 2, 2, 2]
marks_df = pd.DataFrame({'student_id': student_id, 'subject_id': subject_id, 'mark': marks})

In [14]:
marks_df.to_sql('marks', sql, index_label='id', if_exists='replace')

In [15]:
query = """
SELECT students.name as student_name,
subjects.name as subject_name,
marks.mark
FROM marks 
JOIN subjects ON subjects.id = marks.subject_id
JOIN students ON students.id = marks.student_id
"""
marks_from_db = pd.read_sql(query, sql)
marks_from_db.head()

,student_name,subject_name,mark
0,Alice,English,3
1,Alice,English,4
2,Alice,Math,5
3,Bob,English,3
4,Bob,English,5


In [40]:
# средний балл по всем студентам

marks_from_db['mark'].mean()

3.3333333333333335

In [41]:
# средний балл студента по каждому предмету

marks_from_db.groupby(['student_name', 'subject_name']).mean()

mark
student_name subject_name      
Alice        English        3.5
             Math           5.0
Bob          English        4.0
             Math           4.0
Charlie      English        2.0
             Math           2.0

## Задание 3

Сгенерировать данные для таблиц (фунция np.random.randint)

In [16]:
# поставим много оценок

student_ids = np.random.randint(0, 4, size=100)
subject_ids = np.random.randint(0, 3, size=100)
marks = np.random.randint(2, 6, size=100)

more_marks_df = pd.DataFrame({'student_id': student_ids, 'subject_id': subject_ids, 'mark': marks})

# установим индексы от последней записи в БД
last_id = pd.read_sql("SELECT id FROM marks ORDER BY id DESC LIMIT 1", sql).values[0, 0]
more_marks_df.index = list(range(last_id + 1, len(more_marks_df) + last_id + 1))
more_marks_df.head()

,student_id,subject_id,mark
9,1,2,2
10,2,0,2
11,1,1,4
12,0,2,3
13,3,1,3


In [17]:
more_marks_df.to_sql('marks', sql, index_label='id', if_exists='append')

In [18]:
pd.read_sql('SELECT * FROM marks', sql)

,id,student_id,subject_id,mark
0,0,0,0,3
1,1,0,0,4
2,2,0,1,5
3,3,1,0,3
4,4,1,0,5
...,...,...,...,...
104,104,3,0,3
105,105,1,0,2
106,106,3,0,3
107,107,2,2,3


## Задание 4

Добавить таблицу "lessons", установить какая группа в какие дни занималась

Изменить таблицу "marks", связав ее с датой проведения занятия

In [19]:
lesson_dates = ["25-10-2021", "26-10-2021", "27-10-2021", "28-10-2021"]
teacher_to_subject_id = [1, 1, 2, 2]
group_id = [0, 0, 1, 1]

lessons = pd.DataFrame({'date': lesson_dates, 'teacher_to_subject_id': teacher_to_subject_id, 'group_id': group_id})
lessons

,date,teacher_to_subject_id,group_id
0,25-10-2021,1,0
1,26-10-2021,1,0
2,27-10-2021,2,1
3,28-10-2021,2,1


In [20]:
lessons.to_sql('lessons', sql, index_label='id', if_exists='replace')

In [22]:
db_marks_df = pd.read_sql('SELECT * FROM marks', sql)

min_id, max_id = lessons.index[0], lessons.index[-1]
lesson_ids = np.random.randint(min_id, max_id, len(db_marks_df))
db_marks_df['lesson_id'] = lesson_ids
db_marks_df.to_sql('marks', sql, if_exists='replace')

In [27]:
query = """
SELECT t.name as teacher_name,
s.name as subject_name,
g.number as group_number,
l.date as date
FROM lessons as l
JOIN teachers_to_subjects as tts ON tts.id = l.teacher_to_subject_id
JOIN teachers as t ON t.id = tts.teacher_id
JOIN subjects as s ON s.id = tts.subject_id
JOIN groups as g ON g.id = l.group_id
"""
pd.read_sql(query, sql)

,teacher_name,subject_name,group_number,date
0,Bob,IT,101,25-10-2021
1,Bob,IT,101,26-10-2021
2,Charlie,IT,102,27-10-2021
3,Charlie,IT,102,28-10-2021
